# Will It Blend? Deploying BlenderBot

First, load the model.

In [3]:


from parlai.core.agents import create_agent_from_model_file
blender_agent = create_agent_from_model_file("blender_distilled/blender_distilled.pth.checkpoint",
                                            opt_overrides={
                                                "model": "internal:cgen",
                                                "skip_generation": False,
                                                "inference": "delayedbeam",
                                                "beam_delay": 25,
                                                "beam_block_list_filename": "resources/banned_words.txt"
                                            })



06:25:43 | Overriding opt["model"] to internal:cgen (previously: projects.anti_scaling.distillation:DistillTransformerAgent)
06:25:43 | Overriding opt["skip_generation"] to False (previously: True)
06:25:43 | Overriding opt["inference"] to delayedbeam (previously: beam)
06:25:43 | Overriding opt["beam_delay"] to 25 (previously: 30)
06:25:43 | Overriding opt["beam_block_list_filename"] to resources/banned_words.txt (previously: None)
06:25:43 | Using CUDA
06:25:43 | loading dictionary from blender_distilled/blender_distilled.pth.checkpoint.dict
06:25:43 | num words = 8008


In [4]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
blender_agent.opt["temperature"] = 0.7
blender_agent.opt["topp"] = 0.9
blender_agent.opt["topk"] = 5
blender_agent.opt["beam_min_length"] = 15
blender_agent.opt["beam_size"] = 10
blender_agent.opt["inference"] = "beam"
blender_agent

In [30]:
blender_agent.beam_block_list._phrases

{'BYE',
 'BYES',
 'Bye',
 'ByeS',
 'Byes',
 'GOODBYE',
 'GOODBYES',
 'Goodbye',
 'GoodbyeS',
 'Goodbyes',
 'SEE YOU LATER',
 'SEE YOU LATERS',
 'SO LONG',
 'SO LONGS',
 'See You Later',
 'See You LaterS',
 'See you later',
 'See you laters',
 'So Long',
 'So LongS',
 'So long',
 'So longs',
 'bye',
 'byes',
 'goodbye',
 'goodbyes',
 'see you later',
 'see you laters',
 'so long',
 'so longs'}

## Example

Minimal example from Github issue

In [27]:

from itertools import zip_longest
import time
from parlai.utils.strings import normalize_reply

start = time.time()

blender_agent.reset()

history = [
    "",
    "Hi, this is an Alexa Prize Socialbot. I'd like to get to know you a bit better before we chat! Is it all right if I ask for you name?",
    "trenton",
    "Nice to meet you, trenton! I'm looking forward to chatting with you today. So, I think the key to a good mood is eating well. What's a food that always makes you feel good?"
    "hazelnut",
    "I love hazelnut as well! It's nice and crunchy and I especially love it as a topping on my frozen yogurt. What do you like about hazelnut?",
    "um i guess it tastes good",
    "I like the texture and the flavor. I also like that it can be used in so many different cuisines. Do you have a favorite?",
    "I don't know actually",
    "i like all kinds of food but i really like thai and indian food. what about you?",
    "i like mexican",
    "I love Thai food too! I especially like the spicy curry. What is your favorite type of Indian food?",
    "no i like mexican food",
    "i like curry too. i like it with a lot of spices and herbs and vegetables",
    "i guess that sounds good",
    "do you have any favorite curry dishes from indian countries like indian and chinese?",
    "hmm i don't know does lamb curry count",
    "i love indian curry, it is one of the most popular types of curry in the world",
    "you got that right",
    "I love it too. It is so versatile. It can be made with vegetables, meats, and condiments.",
    "for sure yeah",
    "yes it can and it can also be made in a variety of flavors like lamb, beef, pork, chicken, seafood",
    "that's really cool i never heard of seafood curry",
    "that sounds really good. i will have to try it sometime. what do you do for a living?",
    "uh i'm a student",
    "I'm a student right now, but I'm hoping to go back to school in the future.",
    "what?"
]
prefix = None
def process(history):
    pairs = zip_longest(*[iter(history)]*2)

    for user, bot in pairs:
        blender_agent.observe({'text': user, 'episode_done': False})
        print("Added user utterance:", user)
        if bot:
            blender_agent.history.add_reply(bot)
            print("Added bot utterance:", bot)
    response = blender_agent.act(prefix_text=prefix)
    elapsed = time.time() - start
    print(f"BlenderBot replied in {elapsed * 1000:.0f}ms: {normalize_reply(response['text'])}")
    print("All beams:", "\n".join([t for t, s in response["beam_texts"]]))
    print()

    print("BlenderBot's history view:")
    print(blender_agent.history)
    return response
    
response = process(history)

Added user utterance: 
Added bot utterance: Hi, this is an Alexa Prize Socialbot. I'd like to get to know you a bit better before we chat! Is it all right if I ask for you name?
Added user utterance: trenton
Added bot utterance: Nice to meet you, trenton! I'm looking forward to chatting with you today. So, I think the key to a good mood is eating well. What's a food that always makes you feel good?hazelnut
Added user utterance: I love hazelnut as well! It's nice and crunchy and I especially love it as a topping on my frozen yogurt. What do you like about hazelnut?
Added bot utterance: um i guess it tastes good
Added user utterance: I like the texture and the flavor. I also like that it can be used in so many different cuisines. Do you have a favorite?
Added bot utterance: I don't know actually
Added user utterance: i like all kinds of food but i really like thai and indian food. what about you?
Added bot utterance: i like mexican
Added user utterance: I love Thai food too! I especially

In [36]:
response

{'id': 'Cgen',
 'episode_done': False,
 'text': "I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. What about you?",
 'beam_texts': [("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. What about you?",
   -11.454596519470215),
  ("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world!",
   -11.625725746154785),
  ("I love hazelnut! It's nice as it's one of the most widely consumed fruits in the world!",
   -11.740839958190918),
  ("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world.",
   -11.787345886230469),
  ("I love hazelnut! It's nice as it's one of the most widely consumed fruits in the world.",
   -11.881574630737305),
  ("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. Do you like it?",
   -11.947830200195312),
  ("I love hazelnut! It's nice as it's one of the most popular foods in the world!",
   -1

In [18]:
from itertools import zip_longest
history = [
    "hi, where are you from",
    "i'm from the states! i'm from california originally but i live in utah now.",
    "cool, i am also from utah",
    "that's great, what is your favorite place?",
    "um maybe taiwan?"
]
pairs = zip_longest(*[iter(history)]*2)
print(list(pairs))

[('hi, where are you from', "i'm from the states! i'm from california originally but i live in utah now."), ('cool, i am also from utah', "that's great, what is your favorite place?"), ('um maybe taiwan?', None)]


In [14]:
from parlai.agents.transformer.transformer import TransformerGeneratorAgent
